In [ ]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
!git clone https://github.com/konnitiha3/D2NN.git

import sys
sys.path.append('/content/D2NN')

from google.colab import drive
drive.mount('/content/drive')

In [1]:
import time
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from module.two_dim.layers import *

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [3]:
#@title デフォルトのタイトル テキスト
wavelength = 633.0e-9 #@param {type:"number"}
d = 1.0e-6 #@param {type:"number"}
n = 1.5 #@param {type:"number"}

In [4]:
def create_model(**kwargs):
    shape = (100, 100)
    inputs = tf.keras.Input((28, 28))
    phi_max = 2.0 * np.pi
    x = ImageResizing(shape)(inputs)
    x = ImageBinarization(0.5, 0.0, 1.0)(x)
    x = IntensityToElectricField()(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = ElectricFieldToIntensity(normalization="max")(x)
    x = MNISTDetector(10)(x)
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs, x)
    return model

In [6]:
model = create_model()
model.summary()
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
    )

epochs = 10
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    min_delta=0.05,
    patience=2,
)

model_name = "20220608_1"
cholab_path = "/content/drive/MyDrive/D2NN/"
checkpoint_path = cholab_path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

result = model.fit(x_train,
                   y_train,
                   batch_size=64,
                   epochs=epochs,
                   validation_data=(x_test, y_test),
                   callbacks=[cp_callback]
                   )

#path = cholab_path + "trained_model/"+ model_name
path = 'trained_model/test'
model.save(path)

df = pd.DataFrame(result.history)
df.to_csv(path + "/history.csv")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing_1 (ImageResi  (None, 100, 100)         0         
 zing)                                                           
                                                                 
 image_binarization_1 (Image  (None, 100, 100)         0         
 Binarization)                                                   
                                                                 
 intensity_to_electric_field  (None, 100, 100)         0         
 _1 (IntensityToElectricFiel                                     
 d)                                                              
                                                                 
 modulation_1 (Modulation)   (None, 100, 100)          1000

2022-08-10 09:56:06.709228: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


  5/938 [..............................] - ETA: 37:29 - loss: 35.8562 - accuracy: 0.0844

KeyboardInterrupt: 

In [26]:
model = create_model()
model.save("trained_model/test")

INFO:tensorflow:Assets written to: trained_model/test/assets


In [9]:
fig = plt.figure(figsize=(6, 6))
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()
ax1.plot(range(1, epochs + 1), result.history['loss'], color='black', label='Loss')
ax1.set_ylabel('Loss')
ax1.set_xlabel('epoch')
ax2.plot(range(1, epochs + 1), result.history['accuracy'], color='red', label='Accuracy')
ax2.set_ylabel('Accuracy')
# label1と2には、凡例用に各labelのリスト情報が入る
handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
# 凡例をまとめて出力する
ax1.legend(handler1 + handler2, label1 + label2, loc=2, borderaxespad=0.)

In [11]:
model.evaluate(x_test, y_test)

2022-06-04 08:37:49.790385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
